## Error Analysis

In [70]:
import pandas as pd
import numpy as np

In [80]:
# Read in all the files
rule_based_preds = pd.read_json('rule_test_pred.json')
tree_based_preds = pd.read_json('lightgbm_test_pred.json')
svm_preds = pd.read_json('svm_test_pred.json')
bert_prelim_preds = pd.read_json('BERT_test_pred_prelim.json')
bert_final_preds = pd.read_json('BERT_test_pred_final.json')

In [99]:
# How many errors are adjacent classes
lab2ind = {'A1': 0, 'A2': 1, 'B': 2}

def get_class_errors(df, mode='adjacent'):
    ''' Returns a dataframe with the off-by-one errors. '''

    pred_a1 = df['prediction'].map(lambda x: lab2ind[x] == 0)
    pred_a2 = df['prediction'].map(lambda x: lab2ind[x] == 1)
    pred_b = df['prediction'].map(lambda x: lab2ind[x] == 2)

    gold_a1 = df['gold'].map(lambda x: lab2ind[x] == 0)
    gold_a2 = df['gold'].map(lambda x: lab2ind[x] == 1)
    gold_b = df['gold'].map(lambda x: lab2ind[x] == 2)

    adjacent_class_error_df = df[(pred_a1 & gold_a2) | (pred_a2 & gold_a1) | (pred_a2 & gold_b) | (pred_b & gold_a2)]
    two_class_error_df = df[(pred_a1 & gold_b) | (pred_b & gold_a1)]
    
    if mode=='adjacent':
        return adjacent_class_error_df
    elif mode == 'two_class':
        return two_class_error_df
    
def get_pred_over_under(df, mode):
    ''' Returns a dataframe with predictions over or under the correct label, depending on the mode string given. '''
    pred_a1 = df['prediction'].map(lambda x: lab2ind[x] == 0)
    pred_a2 = df['prediction'].map(lambda x: lab2ind[x] == 1)
    pred_b = df['prediction'].map(lambda x: lab2ind[x] == 2)

    gold_a1 = df['gold'].map(lambda x: lab2ind[x] == 0)
    gold_a2 = df['gold'].map(lambda x: lab2ind[x] == 1)
    gold_b = df['gold'].map(lambda x: lab2ind[x] == 2)
    
    if mode=='under':
        return df[(pred_a1 & gold_a2) | (pred_a1 & gold_b) | (pred_a2 & gold_b)]
    elif mode=='over':
        return df[(pred_a2 & gold_a1) | (pred_b & gold_a2) | (pred_b & gold_a1)]
    
    
def print_table(df):
    ''' Prints a table of texts, predictions and correct labels in markdown format
        for easy copying into the report. Only the first 1000 characters in the text are printed,
        for readability. '''
    print('| Text     |      Predicted Label       |  Correct Label |\n|----------|:-------------:|------:|')
    for index, row in df.iterrows():
        print(f"| {row['test_text'][:1000]}... | {row['prediction']} | {row['gold']} |")

### Rule-based predictions

In [72]:
print(rule_based_preds.shape)
rule_based_preds.head()

(32, 3)


,test_text,prediction,gold
0,capítulo —¡paren ya de pelearse! —el hombre al...,A1,A1
1,"¡es con voz de la biblia, o verso de walt whit...",B,B
2,los cuatro hermanos un zapatero tenía cuatro h...,A2,A2
3,una mañana entró un caballero en la tienda de ...,A1,A1
4,había un viejo que tenía una hija muy hermosa....,A1,A1


In [73]:
rule_based_errors = rule_based_preds[rule_based_preds['prediction'] != rule_based_preds['gold']]
print(rule_based_errors.shape)

(15, 3)


Out of 32 texts, the rule-based model gets 15 predictions wrong.

In [86]:
adjacent_class_error_df = get_class_errors(rule_based_errors, mode='adjacent')
two_class_error_df = get_class_errors(rule_based_errors, mode='two_class')

In [87]:
adjacent_class_error.shape

(13, 3)

In [88]:
two_class_error

,test_text,prediction,gold
25,¿en qué mes hablan menos las mujeres?—en el de...,B,A1
29,"¡hola! mi nombre es javier. cuando era niño, m...",A1,B


In [90]:
get_pred_over_under(adjacent_class_error, 'over')

,test_text,prediction,gold
5,ésta es la historia de lola y ana quienes comp...,A2,A1
8,los dos conejos por entre unas matas seguido d...,A2,A1
13,capítulo ii que trata de la primera salida que...,B,A2
14,un día un pollo entra en un bosque. una bellot...,A2,A1
15,ésta es la historia de lola y ana que comparte...,A2,A1
21,"cuba la república de cuba, llamada la ""perla d...",B,A2
22,capítulo —¿por qué no venís a mi casa? —propon...,A2,A1
23,capítulo los siguientes días los amigos hablan...,A2,A1


In [91]:
get_pred_over_under(adjacent_class_error, 'under')

,test_text,prediction,gold
9,hay personas que cifran todo su orgullo en com...,A2,B
11,"en vacaciones, el fin de semana o después del ...",A1,A2
18,"es un poderoso día de verano en misiones, con ...",A2,B
24,"proverbios. (iii) dos amigos de una bolsa, el ...",A1,A2
30,adivinanzas. (i) ¿en qué se parece un esquelet...,A1,A2


13 errors are predictions that have a different of 1.

In [94]:
# Print Table
print_table(rule_based_errors)

| ésta es la historia de lola y ana quienes comparten un piso en barcelona. y sus vecinos, pablo y sam, de norteamérica. a sam le gusta ana. a lola le gusta toby. y a pablo le gustan las nenas, las motocicletas ¡y la pizza! ¡no se mueva! ¡extra! ya empieza … pablo: ¡ya casi está! así, muy bien. ¡perfecto! y ahora, ¡al salón! ahí, ¡perfecto! ¡qué guay, tío! y ahora, algo de beber. coca-cola. y … ¡patatas fritas! ah. y finalmente, ¡radio pizza! ¡sólo quedan dos horas! lola: ¡pablo! ¡¿qué hace tu tele en mi cuarto de baño?! pablo: ah, hola, lola, es que … es que no quería perderme nada. lola: ¿perderte qué? pablo: sí, ya sabes, tengo que … cuando empiece el partido. lola: ¡el partido! pablo: es muy importante. ¡es la copa del mundo! lola: ¡la copa del mundo! ¡estoy harta de la copa del mundo! ¡siempre hablando de la copa del mundo! ¿quién juega hoy? pablo: es la semifinal. ¡españa! lola: ¿contra? sam: ¡américa! pablo: ¡eh, sam! bonito traje. ¡¡pero equipo equivocado!! yo me he puesto mis 

Observations: most of the errors are of a difference of 1 class, meaning that the model still does really well considering that the difference might be ambiguous.

### Tree-Based Predictions

In [81]:
print(tree_based_preds.shape)
tree_based_preds.head()

(32, 3)


,test_text,prediction,gold
0,capítulo —¡paren ya de pelearse! —el hombre al...,A1,A1
1,"¡es con voz de la biblia, o verso de walt whit...",B,B
2,los cuatro hermanos un zapatero tenía cuatro h...,A2,A2
3,una mañana entró un caballero en la tienda de ...,A1,A1
4,había un viejo que tenía una hija muy hermosa....,A2,A1


In [82]:
tree_based_errors = tree_based_preds[tree_based_preds['prediction'] != tree_based_preds['gold']]
print(tree_based_errors.shape)

(12, 3)


In [95]:
# How many errors are adjacent classes
adjacent_class_errors = get_class_errors(tree_based_errors, 'adjacent')
two_class_errors = get_class_errors(tree_based_errors, 'two_class')

In [96]:
adjacent_class_errors

,test_text,prediction,gold
4,había un viejo que tenía una hija muy hermosa....,A2,A1
10,en extremadura vivía un hombre. el hombre era ...,A2,A1
11,"en vacaciones, el fin de semana o después del ...",A1,A2
21,"cuba la república de cuba, llamada la ""perla d...",B,A2
22,capítulo —¿por qué no venís a mi casa? —propon...,A2,A1
23,capítulo los siguientes días los amigos hablan...,A2,A1
24,"proverbios. (iii) dos amigos de una bolsa, el ...",A1,A2
29,"¡hola! mi nombre es javier. cuando era niño, m...",A2,B
30,adivinanzas. (i) ¿en qué se parece un esquelet...,B,A2


In [97]:
two_class_errors

,test_text,prediction,gold
8,los dos conejos por entre unas matas seguido d...,B,A1
19,al partir ¡perla del mar! ¡estrella de occiden...,A1,B
25,¿en qué mes hablan menos las mujeres?—en el de...,B,A1


In [100]:
print_table(tree_based_errors)

| Text     |      Predicted Label       |  Correct Label |
|----------|:-------------:|------:|
| había un viejo que tenía una hija muy hermosa. él era muy ignorante y no sabía lo que era ni oro ni plata. todos los días iba el viejo al bosque a cortar leña. llevaba la leña a la ciudad y la cambiaba por comida para su mujer y su hija. un día estaba cortando el tronco de un árbol grande y oyó lamentos adentro. luego salió un negro muy feo y le dijo: —me has herido y morirás por esto. el viejo se excusó, diciendo: —señor, perdóneme vd. soy muy pobre y vengo a buscar leña para mantener a mi mujer y a mi hija. —¿y es hermosa tu hija?—dijo el negro. —¡oh! sí, señor—dijo el viejo;—y mucho. —pues bien—le dijo el negro,—yo te perdono la vida si me das tu hija por esposa; y si no, morirás. dentro de ocho días te presentarás aquí con la contestación. si la niña quiere o no quiere, debes venir para decírmelo. y ahora abre el tronco de este árbol y hallarás mucho oro. puedes llevarlo a tu mujer y a

### SVM

In [103]:
print(svm_preds.shape)
svm_preds.head()

(32, 3)


,test_text,prediction,gold
0,capítulo —¡paren ya de pelearse! —el hombre al...,A1,A1
1,"¡es con voz de la biblia, o verso de walt whit...",B,B
2,los cuatro hermanos un zapatero tenía cuatro h...,A2,A2
3,una mañana entró un caballero en la tienda de ...,A1,A1
4,había un viejo que tenía una hija muy hermosa....,A1,A1


In [104]:
svm_errors = svm_preds[svm_preds['prediction'] != svm_preds['gold']]
print(svm_errors.shape)

(7, 3)


In [106]:
# How many errors are adjacent classes
adjacent_class_errors = get_class_errors(svm_errors, 'adjacent')
two_class_errors = get_class_errors(svm_errors, 'two_class')

In [107]:
adjacent_class_errors

,test_text,prediction,gold
10,en extremadura vivía un hombre. el hombre era ...,A2,A1
11,"en vacaciones, el fin de semana o después del ...",B,A2
24,"proverbios. (iii) dos amigos de una bolsa, el ...",B,A2
29,"¡hola! mi nombre es javier. cuando era niño, m...",A2,B


In [108]:
two_class_errors

,test_text,prediction,gold
8,los dos conejos por entre unas matas seguido d...,B,A1
14,un día un pollo entra en un bosque. una bellot...,B,A1
25,¿en qué mes hablan menos las mujeres?—en el de...,B,A1


In [109]:
print_table(svm_errors)

| Text     |      Predicted Label       |  Correct Label |
|----------|:-------------:|------:|
| los dos conejos por entre unas matas seguido de perros (no diré corría) volaba un conejo. de su madriguera salió un compañero, y le dijo: "tente, amigo, ¿qué es esto?" "¿qué ha de ser?" responde: "sin aliento llego... dos pícaros galgos me vienen siguiendo." "sí (replica el otro), por allí los veo... pero no son galgos." "pues ¿qué son?"—"podencos." "¡qué! ¿podencos dices? sí, como mi abuelo. galgos y muy galgos: bien visto los tengo." "son podencos: vaya, que no entiendes de eso.' "son galgos, te digo." "digo que podencos." en esta disputa llegando los perros, pillan descuidados a mis dos conejos. los que por cuestiones de poco momento dejan lo que importa, llévense este ejemplo. no debemos detenernos en cuestiones frívolas, olvidando el asunto principal.... | B | A1 |
| en extremadura vivía un hombre. el hombre era rico. era muy rico. tenía casas y viñas, vacas y ovejas, caballos y cerdo

### Neural Models

#### Preliminary BERT model

In [113]:
print(bert_prelim_preds.shape)
bert_prelim_preds.columns = ['test_text', 'prediction', 'gold']
bert_prelim_preds.head()

(32, 3)


,test_text,prediction,gold
0,CAPÍtULO 7\n\n—¡Paren ya de pelearse! —el homb...,A1,A1
1,"¡Es con voz de la Biblia, o verso de Walt Whit...",B,B
2,39. LOS CUATRO HERMANOS\nUn zapatero tenía cua...,A2,A2
3,Una mañana entró un caballero en la tienda de ...,A1,A1
4,Había un viejo que tenía una hija muy hermosa....,A1,A1


In [114]:
bert_prelim_errors = bert_prelim_preds[bert_prelim_preds['prediction'] != bert_prelim_preds['gold']]
print(bert_prelim_errors.shape)

(2, 3)


In [117]:
adjacent_class_errors = get_class_errors(bert_prelim_errors, 'adjacent')
two_class_errors = get_class_errors(bert_prelim_errors, 'two_class')

In [118]:
adjacent_class_errors

,test_text,prediction,gold
11,"En vacaciones, el fin de semana o después del ...",A1,A2
18,"Es un poderoso día de verano en Misiones, con ...",A2,B


In [119]:
two_class_errors

,test_text,prediction,gold


In [120]:
print_table(bert_prelim_errors)

| Text     |      Predicted Label       |  Correct Label |
|----------|:-------------:|------:|
| En vacaciones, el fin de semana o después del trabajo tienes tiempo libre para hacer ejercicio, llevar un pasatiempo o salir a divertirte. Si eres deportista y te gusta el agua puedes nadar, si te gusta la nieve puedes esquiar. Si no dispones de mucho tiempo puedes correr o ir en bicicleta. Hay quien prefiere quedarse en casa a ver la televisión, jugar videojuegos o navegar en Internet. Otras actividades para hacer dentro de casa cuando llueve o hace mucho frío son escuchar tu música favorita, leer un buen libro o aprender a tocar un instrumento. Puedes aprovechar tu tiempo libre para estudiar. Si el tiempo es bueno, tal vez quieras ir de compras, salir con amigos a divertirse o pasar tiempo con la familia en algún lugar divertido para todos, como un parque de diversiones, el teatro o el cine. Aprovechar el tiempo libre es importante para tener una vida feliz.... | A1 | A2 |
| Es un podero

#### Final neural model

In [121]:
print(bert_final_preds.shape)
bert_final_preds.columns = ['test_text', 'prediction', 'gold']
bert_final_preds.head()

(139, 3)


,test_text,prediction,gold
0,CAPÍtULO 7\n\n—¡Paren ya de pelearse! —el homb...,A1,A1
1,.? —pregunta el hombre alto.\n—Es que... —empi...,A1,A1
2,uerta abierta de la choza...\n\n—¿Estás bien? ...,A1,A1
3,"¡Es con voz de la Biblia, o verso de Walt Whit...",A2,B
4,"tiene sangre indígena,\nque aún reza a Jesucri...",B,B


In [122]:
bert_final_errors = bert_final_preds[bert_final_preds['prediction'] != bert_final_preds['gold']]
print(bert_final_errors.shape)

(8, 3)


In [126]:
adjacent_class_errors = get_class_errors(bert_final_errors, 'adjacent')
two_class_errors = get_class_errors(bert_final_errors, 'two_class')

In [127]:
adjacent_class_errors

,test_text,prediction,gold
3,"¡Es con voz de la Biblia, o verso de Walt Whit...",A2,B
10,Una mañana entró un caballero en la tienda de ...,A2,A1
44,"En vacaciones, el fin de semana o después del ...",A1,A2
59,"l ser coronista desta peregrina historia, ruég...",B,A2
99,"Es un poderoso día de verano en Misiones, con ...",A2,B
101,", que cierra con cuidado.\n\n—Vuelve a la hora...",A2,B
115,"…\n—Bueno, solo por curiosidad.\nEl vídeo dura...",A2,A1
135,"¡Hola! Mi nombre es Javier. Cuando era niño, m...",A2,B


In [128]:
two_class_errors

,test_text,prediction,gold


Some of the same errors as in the preliminary model since they are using the same 256-token chunk. Also, cool that the errors are not on the entire text but on a 256-token chunk. Percentage error: only 8/139 = 5.75%, compared to 2/32 = 6.25% on the preliminary model. 

In [129]:
print_table(bert_final_errors)

| Text     |      Predicted Label       |  Correct Label |
|----------|:-------------:|------:|
| ¡Es con voz de la Biblia, o verso de Walt Whitman,
que habría de llegar hasta ti, Cazador!
Primitivo y moderno, sencillo y complicado,
con un algo de Washington y cuatro de Nemrod.
 Eres los Estados Unidos, eres el futuro invasor
de la América ingenua que ... | A2 | B |
| Una mañana entró un caballero en la tienda de un prendero.
Él sacó un cuadro y dijo con cortesía:
—Voy ahora a la oficina. ¿Hará Vd. el favor de guardarme
este cuadro? Lo recogeré por la tarde cuando vuelva a
casa.                                       ... | A2 | A1 |
| En vacaciones, el fin de semana o después del trabajo tienes tiempo libre para hacer ejercicio, llevar un pasatiempo o salir a divertirte. Si eres deportista y te gusta el agua puedes nadar, si te gusta la nieve puedes esquiar. Si no dispones de mucho tiempo puedes correr o ir en bicicleta. Hay quien prefiere quedarse en casa a ver la televisión, jugar vid